## **Description: This demo shows how context engineering can dramatically improve AI responses. It compares answers from a language model asked the same question twice — once without any background information, and once with relevant details retrieved from a small dataset about flight and meeting info. The demo uses vector search to find the right context, then feeds it to the AI to generate smarter, more accurate answers.**

In [2]:
!pip install -q langchain-community
!pip install -q langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.9 MB/s eta 0:00:00


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI API KEY>"
print ("OpenAI API key imported successfully!")

OpenAI API key imported successfully!


## **Import necessary libraries and tools**

In [4]:
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

## **Creating our “knowledge base”**

In [5]:
docs = [
    "Riya's flight to Dallas is on Aug 15 at 10 AM from Newark Liberty International Airport.",
    "Her preferred seat is window, and she usually checks in 24 hours before departure.",
    "She has a meeting in NYC on Aug 16 at 2 PM."
]

## **Creating vector database**

In [6]:
embeddings = OpenAIEmbeddings()
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
split_docs = splitter.create_documents(docs)
vectorstore = FAISS.from_documents(split_docs, embeddings)

## **LLM Setup**

In [ ]:
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever=vectorstore.as_retriever(), memory=memory
)

## **Comparing “with” and “without” context**

In [7]:
# ---- Function for Gradio ----
def compare_outputs(user_query):
    no_context = llm.invoke(user_query).content
    with_context = qa_chain.invoke({"question": user_query})["answer"]
    return no_context, with_context

/tmp/ipython-input-573039160.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


## **User Interface**

In [ ]:
# ---- Gradio UI ----
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Custom CSS for Clear button
    gr.HTML("""
    <style>
      #clear-btn button {
        background-color: #ff4d4f !important;
        color: white !important;
        border: none !important;
        font-weight: bold !important;
        transition: background-color 0.3s ease !important;
      }
      #clear-btn button:hover {
        background-color: #d9363e !important;
      }
    </style>
    """)

    gr.Markdown(
        "<h2 style='text-align:center'>From Guesswork to Genius</h2>"
        "<p style='text-align:center'>Watch an AI agent go from “I don’t know” to “I’ve got this”</p>"
    )

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Ask a question", placeholder="Ask a question")
            with gr.Row():
                submit_btn = gr.Button("Get Answers", variant="primary")
                clear_btn = gr.Button("Clear", elem_id="clear-btn")

        with gr.Column():
            no_context_box = gr.Textbox(label="Without Context", lines=4)
            with_context_box = gr.Textbox(label="With Context", lines=4)

    submit_btn.click(compare_outputs, inputs=user_input, outputs=[no_context_box, with_context_box])
    clear_btn.click(lambda: ("", "", ""), outputs=[user_input, no_context_box, with_context_box])

demo.launch()